In [12]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
MODEL = "llama2"

In [13]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

"\nWhy don't scientists trust atoms? Because they make up everything! 😂"

In [14]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

'Why was the math book sad? Because it had too many problems!'

In [15]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("mlschool.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='Building Machine Learning Systems That Don\'t Suck\nA live, interactive program that\'ll help you build production-readymachine learning\nsystems from the ground up.\nNext cohort:\xa0July1 - 18, 2024\nCheck the schedulefor more details about upcoming cohorts.\nI want to join!Sign in\nLearn how to design, build, deploy, and scale machine learning\nsystems to solve real-world problems.\nI\'ll lose my mind if I see another book or course teaching people the same basic ideas\nfor the hundredth time. Most people are stuck in beginner mode, and finding help to\nsolve real-world problems is hard.\nI want to change that.\nI started writing software 30 years ago. I\'ve written pipelines and trained models for\nsome of the largest companies in the world. I want to show you how to do the same.\nThis is the class I wish I had taken when I started."This is the best machine learning course I\'ve done.\nWorth every cent."\n— Jose Reyes, AI/ML at Cevo Australia6/23/24, 2:22 PM 

In [16]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



In [17]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

' Sure! Based on the context you provided, my answer would be:\n\n"My name is Santiago."'

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch #vectorstore to store the embeddings of the pdf

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [8]:
retriever = vectorstore.as_retriever() #retriever component of langchain
retriever.invoke("machine learning") #gives the top 4 concepts matching with machine learning

[Document(page_content="You'll design and write the code to build an end-to-end machine learning\nsystem starting from scratch.\nYou'll learn best practices to tackle the most significant challenges machine\nlearning engineers face to build, evaluate, run, monitor, and maintain machine\nlearning systems in real-world scenarios.\nYou'll learn how to use techniques like active learning, distributed training,\nadversarial validation, human-in-the-loop deployments, model compression,\ntest-time augmentation, testing in production, among many others.\nYou'll learn how to create training, deploying, monitoring, and inference\npipelines using Amazon SageMaker and open-source tools.\nForget about theoretical concepts. This program will show you some of the things I've\nlearned from real-life examples I've built during more than 30 years in the industry.\nCheck the program syllabus\xa0→\nReal-life examples and case studies\nLearn from practical experience building machine\nlearning systems that

In [9]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [11]:
questions = [
    "What is the purpose of the course?",
    "How many hours of live sessions?",
    "How many coding assignments are there in the program?",
    "Is there a program certificate upon completion?",
    "What programming language will be used in the program?",
    "How much does the program cost?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the purpose of the course?
Answer: The purpose of the course is to teach practical skills and insights that will help individuals build machine learning systems that work in the real world.

Question: How many hours of live sessions?
Answer: Each iteration of the program offers 18 hours of live training spread over 3 weeks.

Question: How many coding assignments are there in the program?
Answer: There are 100 coding assignments in the program.

Question: Is there a program certificate upon completion?
Answer: Yes, there is a program certificate upon completion.

Question: What programming language will be used in the program?
Answer: Python

Question: How much does the program cost?
Answer: The program costs $450.

